<a href="https://colab.research.google.com/github/ShubhamMhatre2727/ABCcalculator/blob/main/naive_custome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as nlp

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

import warnings

import re
import string
import pickle

tweets = pd.read_csv("/content/drive/MyDrive/csvData/training.1600000.processed.noemoticon.csv", encoding='latin', names=['sentiment', 'id','date', 'query', 'user', 'tweet'])
tweets['sentiment'] = tweets['sentiment'].replace(4, 1)
tweets.drop(['id', 'date', 'query', 'user'], axis=1, inplace=True)

(tweets.isnull().sum() / len(tweets))*100
tweets['tweet'] = tweets['tweet'].astype('str')

positives = tweets['sentiment'][tweets.sentiment == 1]
negatives = tweets['sentiment'][tweets.sentiment == 0]

stopword = set(stopwords.words('english'))

warnings.filterwarnings("ignore", category=DeprecationWarning)

def remove_pattern(text,pattern):
    
    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)
    
    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)
    
    return text

tweets['Tidy_Tweets'] = tweets['tweet'].str.replace("http\S+", '')
tweets['Tidy_Tweets'] = np.vectorize(remove_pattern)(tweets['Tidy_Tweets'], "@[\w]*")
tweets['Tidy_Tweets'] = tweets['Tidy_Tweets'].str.replace("[^a-zA-Z#]", " ")
tweets['Tidy_Tweets'] = tweets['Tidy_Tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

tokenized_tweet = tweets['Tidy_Tweets'].apply(lambda x: x.split())

tokenized_tweet.to_csv("tokenized_tweets.csv")

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

from sklearn.model_selection import train_test_split

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(tweets['Tidy_Tweets'].values.astype('U'))

X=text_counts
y=tweets['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=19)
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from math import *
cnb = ComplementNB()
cnb.fit(X_train, y_train)
cross_cnb = cross_val_score(cnb, X, y,n_jobs = -1)
print("Cross Validation score = ",cross_cnb)                
print ("Train accuracy ={:.2f}%".format(cnb.score(X_train,y_train)*100))
print ("Test accuracy ={:.2f}%".format(cnb.score(X_test,y_test)*100))
train_acc_cnb=cnb.score(X_train,y_train)
test_acc_cnb=cnb.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version.


Cross Validation score =  [0.74629375 0.73945313 0.7403625  0.74831563 0.74181562]
Train accuracy =78.00%
Test accuracy =75.02%


Cross Validation score =  [0.74775625 0.740425   0.74175313 0.75003438 0.74280625]
Train accuracy =78.14%
Test accuracy =75.13%


In [ ]:
review = "working on it"

review_vector = cv.transform([review]) # vectorizing
print(cnb.predict(review_vector))


# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
score = SentimentIntensityAnalyzer().polarity_scores(review)
print(score)

# positive sentiment : (compound score >= 0.05) 
# neutral sentiment : (compound score > -0.05) and (compound score < 0.05) 
# negative sentiment : (compound score <= -0.05)

# 1 --> Positive
# 0 --> Negative

[0]
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
